<h1 style='text-align: center'>Cramerton Parks and Rec Department Address Data Cleaning</h1>

This notebook will help clean and transform the parcel data to meet the software requirements for tracking in-town and out-of-town residents.

In [1]:
#Import Required Libraries
import pandas as pd
import numpy as np
import re

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [2]:
# Read the main parcels CSV file
df = pd.read_csv('Cramerton_Parcels.csv')

# Read the ETJ IDs CSV file
etj_ids = pd.read_csv('ETJ_IDS.csv')

# Display initial count
print(f"Initial number of records: {len(df)}")

# Remove records that match ETJ IDs
df = df[~df['AKPAR'].isin(etj_ids['AKPAR'])]

# Display count after removal
print(f"Number of records after removing ETJ parcels: {len(df)}")

# Display basic information about the cleaned dataset
print("\nDataset Info:")
print(df.info())

print("\nFirst few rows:")
print(df.head())

print("\nColumn names in the dataset:")
print(df.columns.tolist())

Initial number of records: 3058
Number of records after removing ETJ parcels: 2899

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 2899 entries, 38 to 3052
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   AKPAR       2899 non-null   int64 
 1   MASTER      2899 non-null   object
 2   WHOLE_ADDR  2899 non-null   object
 3   POSTAL      2899 non-null   object
 4   STATE       2899 non-null   object
 5   ZIP         2899 non-null   int64 
 6   JAN1_NAME1  2899 non-null   object
 7   JAN1_NAME2  2899 non-null   object
 8   PID         2899 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 226.5+ KB
None

First few rows:
     AKPAR MASTER           WHOLE_ADDR     POSTAL STATE    ZIP  \
38  185570    Yes        245 MARKET ST  CRAMERTON    NC  28032   
40  185575    Yes          3 GROVES ST  CRAMERTON    NC  28032   
41  185576    Yes        243 MARKET ST  CRAMERTON    NC  28032   
42  185729    Yes  593

In [3]:
# Count of rows with empty values
print("\nInitial row counts:")
print(f"Total rows: {len(df)}")
print(f"Empty strings: {len(df[df['WHOLE_ADDR'] == ''])}")
print(f"Whitespace only: {len(df[df['WHOLE_ADDR'].str.isspace()])}")
print(f"Null values: {len(df[df['WHOLE_ADDR'].isna()])}")

# Remove empty, whitespace, and null values
df['WHOLE_ADDR'] = df['WHOLE_ADDR'].str.strip()
df = df[
    (df['WHOLE_ADDR'] != '') & 
    (df['WHOLE_ADDR'].notna()) & 
    (~df['WHOLE_ADDR'].str.isspace())
]

# Verification counts
print("\nFinal row count:", len(df))

# Most common addresses
print("\nTop 10 most common addresses:")
print(df['WHOLE_ADDR'].value_counts().head(10))

# Save cleaned data to CSV
output_file = 'Cramerton_Empty_Values_Removed.csv'
df.to_csv(output_file, index=False)
print(f"\nCleaned data saved to {output_file}")


Initial row counts:
Total rows: 2899
Empty strings: 0
Whitespace only: 216
Null values: 0

Final row count: 2683

Top 10 most common addresses:
WHOLE_ADDR
116 HARRISON DR        1
245 MARKET ST          1
3 GROVES ST            1
243 MARKET ST          1
5933 WILKINSON BLVD    1
5939 WILKINSON BLVD    1
5945 WILKINSON BLVD    1
5932 WILKINSON BLVD    1
5938 WILKINSON BLVD    1
111 HUBBARD ST         1
Name: count, dtype: int64

Cleaned data saved to Cramerton_Empty_Values_Removed.csv


In [4]:
# Create new DataFrame with updated column names while keeping existing data
output_df = pd.DataFrame(columns=[
    'AKPAR',  # Keeping the original AKPAR column
    'ValidationType',
    'AddressLine1',
    'AddressLine2',
    'StreetNumberFrom',
    'StreetNumberTo',
    'City',
    'State',
    'ZipCode'
])

# Copy data from existing DataFrame to new structure
output_df['AKPAR'] = df['AKPAR']
output_df['AddressLine1'] = df['WHOLE_ADDR']
output_df['City'] = df['POSTAL']
output_df['State'] = df['STATE']
output_df['ZipCode'] = df['ZIP']

# Set default values for new columns
output_df['ValidationType'] = 'E'  # Default to exact match
output_df['AddressLine2'] = ''     # Always blank
output_df['StreetNumberFrom'] = '' # Will be blank for exact matches
output_df['StreetNumberTo'] = ''   # Will be blank for exact matches

# Save this intermediate step to CSV
output_df.to_csv('Cramerton_New_Columns.csv', index=False)

# Display verification information
print(f"\nNumber of rows in new DataFrame: {len(output_df)}")
print("\nColumns in the new DataFrame:")
print(output_df.columns.tolist())
print("\nSample of the new structure:")
print(output_df.head())
print("\nFile has been saved as 'Cramerton_Addresses_Step1.csv'")


Number of rows in new DataFrame: 2683

Columns in the new DataFrame:
['AKPAR', 'ValidationType', 'AddressLine1', 'AddressLine2', 'StreetNumberFrom', 'StreetNumberTo', 'City', 'State', 'ZipCode']

Sample of the new structure:
     AKPAR ValidationType         AddressLine1 AddressLine2 StreetNumberFrom  \
38  185570              E        245 MARKET ST                                 
40  185575              E          3 GROVES ST                                 
41  185576              E        243 MARKET ST                                 
42  185729              E  5933 WILKINSON BLVD                                 
43  185730              E  5939 WILKINSON BLVD                                 

   StreetNumberTo       City State  ZipCode  
38                 CRAMERTON    NC    28032  
40                 CRAMERTON    NC    28032  
41                 CRAMERTON    NC    28032  
42                   BELMONT    NC    28012  
43                   BELMONT    NC    28012  

File has been sa

In [5]:
# Handle special cases for South New Hope Road, North Main Street, and South Main Street
def standardize_special_streets(address):
    if 'S NEW HOPE RD' in address:
        # Keep the house number and standardize the rest
        parts = address.split()
        if parts[0].isdigit():
            return f"{parts[0]} SOUTH NEW HOPE RD"
        return 'SOUTH NEW HOPE RD'
    elif 'N MAIN ST' in address:
        return 'NORTH MAIN ST'
    elif 'S MAIN ST' in address:
        return 'SOUTH MAIN ST'
    return address

def clean_street_name(address):
    parts = address.split()
    if not parts:
        return ""
    
    # Remove house number if present
    if parts[0].isdigit():
        parts = parts[1:]
    
    # Remove single letter prefixes (A, B, C, D, etc.)
    if parts and len(parts[0]) == 1 and parts[0].isalpha():
        parts = parts[1:]
    
    return ' '.join(parts)

# Dictionary of residential streets with their city, state, and zip
residential_streets_info = {
    'NORTH MAIN ST': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'SOUTH MAIN ST': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'HIDDEN PASTURES DR': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'HAVEN CIR': {'city': 'BELMONT', 'state': 'NC', 'zip': 28012},
    'BOATHOUSE LN': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'MISTY HARBOR CIR': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'RIVERBED LN': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'CRAMERTON VILLAGE DR': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'WATERSIDE LN': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'SEVENTH ST': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'CENTER ST': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'DAWSON ST': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'ARMSTRONG DR': {'city': 'BELMONT', 'state': 'NC', 'zip': 28012},
    'EIGHTH AVE': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'BROOKLYN AVE': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'WASHINGTON ST': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'WOODLAWN AVE': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'MAYFLOWER AVE': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032},
    'MARKET ST': {'city': 'CRAMERTON', 'state': 'NC', 'zip': 28032}
}

# Dictionary for numbered streets
numbered_streets = {
    'FIRST': '1ST',
    'SECOND': '2ND',
    'THIRD': '3RD',
    'FOURTH': '4TH',
    'FIFTH': '5TH',
    'SIXTH': '6TH',
    'SEVENTH': '7TH',
    'EIGHTH': '8TH',
    'NINTH': '9TH',
    'TENTH': '10TH',
    'ELEVENTH': '11TH',
    'TWELFTH': '12TH',
    'THIRTEENTH': '13TH',
    'FOURTEENTH': '14TH',
    'FIFTEENTH': '15TH',
    'SIXTEENTH': '16TH',
    'SEVENTEENTH': '17TH',
    'EIGHTEENTH': '18TH'
}

# Create a copy of the current DataFrame
df_working = output_df.copy()

# Standardize the special street names
df_working['AddressLine1'] = df_working['AddressLine1'].apply(standardize_special_streets)

# Extract clean street names for all records
df_working['clean_street'] = df_working['AddressLine1'].apply(clean_street_name)

# Identify records to remove (where clean street matches residential streets)
records_to_remove = df_working[df_working['clean_street'].isin(residential_streets_info.keys())]

# Create new records for residential streets with correct city/state/zip
residential_records = pd.DataFrame()
for street, info in residential_streets_info.items():
    new_row = pd.DataFrame({
        'AKPAR': [''],  # Empty AKPAR for new residential records
        'ValidationType': ['S'],
        'AddressLine1': [street],
        'AddressLine2': [''],
        'StreetNumberFrom': [''],
        'StreetNumberTo': [''],
        'City': [info['city']],
        'State': [info['state']],
        'ZipCode': [info['zip']]
    })
    residential_records = pd.concat([residential_records, new_row])

# Handle numbered streets
numbered_records = pd.DataFrame()
for full_name, short_name in numbered_streets.items():
    # Create two records for each numbered street
    for street_name in [f"{full_name} ST", f"{short_name} ST"]:
        new_row = pd.DataFrame({
            'AKPAR': [''],
            'ValidationType': ['S'],
            'AddressLine1': [street_name],
            'AddressLine2': [''],
            'StreetNumberFrom': [''],
            'StreetNumberTo': [''],
            'City': ['CRAMERTON'],
            'State': ['NC'],
            'ZipCode': [28032]
        })
        numbered_records = pd.concat([numbered_records, new_row])

# Remove the residential street records from the main DataFrame
df_working = df_working[~df_working['clean_street'].isin(residential_streets_info.keys())]

# Remove numbered street records
for full_name, short_name in numbered_streets.items():
    df_working = df_working[
        ~df_working['clean_street'].str.contains(full_name, case=False, na=False) &
        ~df_working['clean_street'].str.contains(short_name, case=False, na=False)
    ]

# Combine all records
final_df = pd.concat([df_working, residential_records, numbered_records])

# Drop only the clean_street column from final output
final_df = final_df.drop(['clean_street'], axis=1, errors='ignore')
records_to_remove = records_to_remove.drop(['clean_street'], axis=1, errors='ignore')

# Save both files
final_df.to_csv('Cramerton_Final.csv', index=False)
records_to_remove.to_csv('Removed_Records.csv', index=False)

# Print summary
print(f"\nRecords removed: {len(records_to_remove)}")
print(f"New residential records added: {len(residential_records)}")
print(f"New numbered street records added: {len(numbered_records)}")
print(f"Final record count: {len(final_df)}")

print("\nSample of removed records:")
print(records_to_remove[['AKPAR', 'AddressLine1']].head())
print("\nSample of new numbered street records:")
print(numbered_records[['ValidationType', 'AddressLine1', 'City', 'State', 'ZipCode']].head())


Records removed: 600
New residential records added: 19
New numbered street records added: 36
Final record count: 1957

Sample of removed records:
      AKPAR   AddressLine1
38   185570  245 MARKET ST
41   185576  243 MARKET ST
264  194641  200 MARKET ST
266  194643  202 MARKET ST
267  194644  210 MARKET ST

Sample of new numbered street records:
  ValidationType AddressLine1       City State  ZipCode
0              S     FIRST ST  CRAMERTON    NC    28032
0              S       1ST ST  CRAMERTON    NC    28032
0              S    SECOND ST  CRAMERTON    NC    28032
0              S       2ND ST  CRAMERTON    NC    28032
0              S     THIRD ST  CRAMERTON    NC    28032
